# 교차분석(카이제곱 검정)
| 설명변수, 반응변수가 범주형 변수인 경우에 사용하는 통계 기법

```scipy.stats.chisquare(관찰 빈도(value_counts), 기대빈도, 자유도 조정, 축)```
* 데이터에서 각 범주에 따른 **결과변수의 분포**를 설명하거나,
    
    범주형 변수가 두 개 이상인 경우 **두 변수가 상관이 있는지**를 검정하는 것이 목적
* 교차표를 통해 각 셀의 관찰빈도와 기대빈도 간의 차이를 검정하는 기법이다.
    * 관찰빈도: 자료로부터 얻은 빈도분포
    * 기대빈도: 두 변수가 독립일 때 이론적으로 기대할 수 있는 빈도 분포
    
<img src='./imgs/crosstable.jpeg' width='440px' height='120px'></img>

* 목적에 따라 적합성, 독립성, 동질성 검정으로 나뉨
    1. 적합성 검정: 각 범주에 따른 데이터의 빈도분포가 **이론적으로 기대하는 분포를 따르는가?**
    
        (주사위값의 각 확률이 1/6이 맞는가?)
    2. 독립성 검정: 모집단이 두 개의 변수 A, B에 의해 범주화되었을 때, **두 변수들의 관계가 독립인가?**
    
        (비만 유무와 질환 유무가 주어졌을 때, 비만에 따른 질환 비율의 차이가 존재하는가?)
    3. 동질성 검정: 모집단이 임의의 변수에따라 R개의 속성으로 범주화되었을 때, R개의 부분 모집단에서 추출한 표본이 C개의 범주화된 집단의 분포와 동일한가?

In [1]:
import scipy.stats as stats
from scipy.stats import chisquare, chi2_contingency # 적합성, 독립성 검정

import pandas as pd

### 카이제곱 적합성 검정 예제 (성별-독립, 생존여부-종속)
* 타이타닉 생존 데이터

    귀무가설: 타이타닉호 생존자 중 남녀 비율이 반반이다.

    대립가설: 타이타닉호 생존자 중 남녀 비율은 반반이 아니다.

In [2]:
# 카이제곱 적합성 검정 예제
data = pd.read_csv("https://raw.githubusercontent.com/ADPclass/ADP_book_ver01/main/data/titanic.csv")
data.info(), data.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   survived     891 non-null    int64  
 1   pclass       891 non-null    int64  
 2   sex          891 non-null    object 
 3   age          714 non-null    float64
 4   sibsp        891 non-null    int64  
 5   parch        891 non-null    int64  
 6   fare         891 non-null    float64
 7   embarked     889 non-null    object 
 8   class        891 non-null    object 
 9   adult_male   891 non-null    bool   
 10  embark_town  889 non-null    object 
dtypes: bool(1), float64(2), int64(4), object(4)
memory usage: 70.6+ KB


(None,
    survived  pclass     sex   age  sibsp  parch     fare embarked  class  \
 0         0       3    male  22.0      1      0   7.2500        S  Third   
 1         1       1  female  38.0      1      0  71.2833        C  First   
 2         1       3  female  26.0      0      0   7.9250        S  Third   
 3         1       1  female  35.0      1      0  53.1000        S  First   
 4         0       3    male  35.0      0      0   8.0500        S  Third   
 
    adult_male  embark_town  
 0        True  Southampton  
 1       False    Cherbourg  
 2       False  Southampton  
 3       False  Southampton  
 4        True  Southampton  )

In [3]:
# 생존자 데이터만 분류
sur = data.loc[data['survived'] == 1]
table = sur['sex'].value_counts()
table
# 유의수준 0.05로 적합도 검정 수행
chi = chisquare(table, f_exp=[171, 171]) ## 관찰빈도, 기대빈도(50:50) 입력
print('적합도 검정\n', chi)

적합도 검정
 Power_divergenceResult(statistic=44.95906432748538, pvalue=2.0119672574477235e-11)


* p-value < 0.05이므로 귀무가설이 기각

    **생존자 중 남녀 비율은 50:50이라고 할 수 없다**

### 카이제곱 독립성 검정 예제
```stats.chi2_contingency(관찰빈도)```

* 관찰빈도는 ```pd.crosstab()``` 결과값을 입력한다
* 4개의 결과값이 Return
    1. chi2: 검정통계량
    2. p: p-value
    3. dof: 자유도
    4. expected: 기대빈도

* 예제) 좌석등급과 생존여부가 서로 독립인가?
    
    귀무가설: 두 변수는 독립이다

    대립가설: 두 변수는 독립이 아니다

In [6]:
# 교차표 만들기(관찰빈도)
table = pd.crosstab(data['class'], data['survived']) # pd.crosstab(Series, Series) 교차표생성
print('교차표\n', table)

# 독립성 검정
chi, p, dof, exp = chi2_contingency(table)
print(f'''
      Statictic: {chi}
      p-value: {p}
      df: {dof}
      expect:\n{exp}''')

교차표
 survived    0    1
class             
First      80  136
Second     97   87
Third     372  119

      Statictic: 102.88898875696056
      p-value: 4.549251711298793e-23
      df: 2
      expect:
[[133.09090909  82.90909091]
 [113.37373737  70.62626263]
 [302.53535354 188.46464646]]


* p-value < 0.05이므로 귀무가설 기각

    **좌석등급과 생존여부는 독립이 아니다**